In [3]:
import torch
from masksembles.torch import Masksembles2D

In [3]:
mask = Masksembles2D(10,4,2.0)

In [4]:
import imgaug.augmenters as iaa
from torchvision import transforms
from pytorchyolo.utils.transforms import ToTensor, PadSquare, RelativeLabels, AbsoluteLabels, ImgAug


class DefaultAug(ImgAug):
    def __init__(self, ):
        self.augmentations = iaa.Sequential([
            iaa.Sharpen((0.0, 0.1)),
            iaa.Affine(rotate=(-0, 0), translate_percent=(-0.1, 0.1), scale=(0.8, 1.5)),
            iaa.AddToBrightness((-60, 40)),
            iaa.AddToHue((-10, 10)),
            iaa.Fliplr(0.5),
        ])


class StrongAug(ImgAug):
    def __init__(self, ):
        self.augmentations = iaa.Sequential([
            iaa.Dropout([0.0, 0.01]),
            iaa.Sharpen((0.0, 0.1)),
            iaa.Affine(rotate=(-10, 10), translate_percent=(-0.1, 0.1), scale=(0.8, 1.5)),
            iaa.AddToBrightness((-60, 40)),
            iaa.AddToHue((-20, 20)),
            iaa.Fliplr(0.5),
        ])


AUGMENTATION_TRANSFORMS = transforms.Compose([
    AbsoluteLabels(),
    DefaultAug(),
    PadSquare(),
    RelativeLabels(),
    ToTensor(),
])

In [20]:
from torchvision import transforms
from pytorchyolo.utils.transforms import ToTensor, PadSquare, RelativeLabels, AbsoluteLabels, ImgAug

a = torch.nn.Sequential(
    transforms.CenterCrop(10),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
)

class DefaultAug(ImgAug):
    def __init__(self, ):
        self.augmentations = torch.nn.Sequential([
            transforms.CenterCrop(10),
            transforms.RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.8, 1.5)),
            transforms.ColorJitter(),
            transforms.RandomHorizontalFlip(0.5),
        ])



AUGMENTATION_TRANSFORMS = torch.nn.Sequential([
    AbsoluteLabels(),
    DefaultAug(),
    PadSquare(),
    RelativeLabels(),
    ToTensor(),
])

TypeError: list is not a Module subclass

In [1]:
import tqdm
from pytorchyolo.utils.datasets import *
from torch.utils.data import DataLoader
import argparse
from pytorchyolo.utils.transforms import DEFAULT_TRANSFORMS

def _create_validation_data_loader(img_path, batch_size, img_size, n_cpu):
    """
    Creates a DataLoader for validation.

    :param img_path: Path to file containing all paths to validation images.
    :type img_path: str
    :param batch_size: Size of each image batch
    :type batch_size: int
    :param img_size: Size of each image dimension for yolo
    :type img_size: int
    :param n_cpu: Number of cpu threads to use during batch generation
    :type n_cpu: int
    :return: Returns DataLoader
    :rtype: DataLoader
    """
    dataset = ListDataset(img_path, img_size=img_size, multiscale=False, transform=DEFAULT_TRANSFORMS)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=n_cpu,
        pin_memory=True,
        collate_fn=dataset.collate_fn)
    return dataloader

parser = argparse.ArgumentParser(description="Trains the YOLO model.")
parser.add_argument("-m", "--model", type=str, default="config/yolov3.cfg", help="Path to model definition file (.cfg)")
parser.add_argument("-d", "--data", type=str, default="config/coco.data", help="Path to data config file (.data)")
parser.add_argument("-e", "--epochs", type=int, default=300, help="Number of epochs")
parser.add_argument("-v", "--verbose", action='store_true', help="Makes the training more verbose")
parser.add_argument("--n_cpu", type=int, default=8, help="Number of cpu threads to use during batch generation")
parser.add_argument("-w", "--pretrained_weights", type=str, default="weights/darknet53.conv.74", help="Path to checkpoint file (.weights or .pth). Starts training from checkpoint model")
parser.add_argument("--checkpoint_interval", type=int, default=1, help="Interval of epochs between saving model weights")
parser.add_argument("--evaluation_interval", type=int, default=1, help="Interval of epochs between evaluations on validation set")
parser.add_argument("--multiscale_training", action="store_true", help="Allow multi-scale training")
parser.add_argument("--iou_thres", type=float, default=0.5, help="Evaluation: IOU threshold required to qualify as detected")
parser.add_argument("--conf_thres", type=float, default=0.1, help="Evaluation: Object confidence threshold")
parser.add_argument("--nms_thres", type=float, default=0.5, help="Evaluation: IOU threshold for non-maximum suppression")
parser.add_argument("--logdir", type=str, default="logs", help="Directory for training log files (e.g. for TensorBoard)")
parser.add_argument("--seed", type=int, default=-1, help="Makes results reproducable. Set -1 to disable.")
args, unknown = parser.parse_known_args()
print(f"Command line arguments: {args}")

def xywh2xyxy(x):
    y = x.new(x.shape)
    y[..., 0] = x[..., 0] - x[..., 2] / 2
    y[..., 1] = x[..., 1] - x[..., 3] / 2
    y[..., 2] = x[..., 0] + x[..., 2] / 2
    y[..., 3] = x[..., 1] + x[..., 3] / 2
    return y

def parse_data_config(path):
    """Parses the data configuration file"""
    options = dict()
    options['gpus'] = '0,1,2,3'
    options['num_workers'] = '10'
    with open(path, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.strip()
        if line == '' or line.startswith('#'):
            continue
        key, value = line.split('=')
        options[key.strip()] = value.strip()
    return options


# Get data configuration
data_config = parse_data_config(args.data)
train_path = data_config["train"]
valid_path = data_config["valid"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

validation_dataloader = _create_validation_data_loader(
        valid_path,
        16,
        416,
        args.n_cpu)

Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

labels = []
sample_metrics = []  # List of tuples (TP, confs, pred)
from torch.autograd import Variable


for _, imgs, targets in tqdm.tqdm(validation_dataloader, desc="Validating"):
    # Extract labels
    labels += targets[:, 1].tolist()
    # Rescale target
    targets[:, 2:] = xywh2xyxy(targets[:, 2:])
    targets[:, 2:] *= 416

    imgs = Variable(imgs.type(Tensor), requires_grad=False)

Command line arguments: Namespace(model='config/yolov3.cfg', data='config/coco.data', epochs=300, verbose=False, n_cpu=8, pretrained_weights='weights/darknet53.conv.74', checkpoint_interval=1, evaluation_interval=1, multiscale_training=False, iou_thres=0.5, conf_thres=0.1, nms_thres=0.5, logdir='logs', seed=-1)


Validating: 100%|██████████| 313/313 [00:25<00:00, 12.29it/s]
